In [ ]:
 import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
# 구글드라이브 연결
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# 데이터파일 불러오기
df = pd.read_csv('/gdrive/My Drive/3-2_machine_learning/image-10k.csv', header=None)

X = df.iloc[:, 1:].values     # 데이터샘플
y = df.iloc[:, 0].values      # 타겟레이블


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1, stratify=y)

images = ['T-shirt', 'Trouser', 'Sweater', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot']

In [ ]:
import cv2

hog = cv2.HOGDescriptor((28, 28), (14, 14), (7, 7), (7, 7), 9)

hog_x_train = np.zeros((5000,324),dtype="float64")
hog_x_test = np.zeros_like(hog_x_train)

for i in range(5000):
  hog_x_train[i]=hog.compute(x_train[i].reshape(28,28).astype(np.uint8)).squeeze()

for i in range(5000):
  hog_x_test[i]=hog.compute(x_test[i].reshape(28,28).astype(np.uint8)).squeeze()

x_train = hog_x_train
x_test = hog_x_test
print(x_test.shape)

(5000, 324)


In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)



In [ ]:
TOP=0
ELSE=1

coarse_classify=np.ones_like(y_train)
test_coarse=np.ones_like(y_test)

coarse_classify[y_train==0]=TOP
coarse_classify[y_train==2]=TOP
coarse_classify[y_train==3]=TOP
coarse_classify[y_train==4]=TOP
coarse_classify[y_train==6]=TOP


test_coarse[y_test==0]=TOP
test_coarse[y_test==2]=TOP
test_coarse[y_test==3]=TOP
test_coarse[y_test==4]=TOP
test_coarse[y_test==6]=TOP

In [ ]:
x_train = x_train[coarse_classify == TOP]
y_train = y_train[coarse_classify == TOP]
x_test = x_test[test_coarse == TOP]
y_test = y_test[test_coarse == TOP]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

'''#0.986
knn = KNeighborsClassifier()

params = {
    'n_neighbors': [3, 5,7,11,13,17],
    'weights': ['uniform', 'distance'],
    'algorithm':['auto','ball_tree','kd_tree','brute'],
    'p':[1,2]
}


grid_tree = GridSearchCV(knn, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, y_train)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
knn = grid_tree.best_estimator_
pred_y_knn = knn.predict(x_test)

#knn.fit(x_train, corse_classify)
#pred_y_knn = knn.predict(x_test)
print('knn 분류기 정확도', accuracy_score(y_test, pred_y_knn))#Voting 분류기 정확도 0.99'''

'''best parameters :  {'algorithm': 'auto', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
best score :  0.7276023119079765
knn 분류기 정확도 0.7276'''


knn = KNeighborsClassifier(weights='distance',algorithm='auto',p=1,n_neighbors=5)
knn.fit(x_train, y_train)
pred_y_knn = knn.predict(x_test)
print('knn 분류기 정확도', accuracy_score(y_test, pred_y_knn))#Voting 분류기 정확도 0.99'

#0.7548



knn 분류기 정확도 0.7368


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'hidden_layer_sizes': [(100,),(100,10),(512,128,32),(512,128),(50,),(50,5)],
    'activation': ['relu'],
    'batch_size':[500,250],
    'solver':['adam'],
    'learning_rate':['adaptive','invscaling'],
    'max_iter':[500,250,1000],
    'alpha':[1e-2,1e-1,1e0,1e1,1e2]
}

grid_tree = GridSearchCV(mlp, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, y_train)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
mlp = grid_tree.best_estimator_
pred_y_mlp = mlp.predict(x_test)

#mlp.fit(x_train, corse_classify)
#pred_y_mlp = mlp.predict(x_test)
print('mlp 분류기 정확도', accuracy_score(y_test, pred_y_mlp))
'''
best parameters :  {'activation': 'relu', 'alpha': 0.01, 'batch_size': 500, 'hidden_layer_sizes': (512, 128), 'learning_rate': 'adaptive', 'max_iter': 250, 'solver': 'adam'}
best score :  0.7828047286060708
mlp 분류기 정확도 0.7908
'''






'''
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', batch_size=500,
                    solver='adam', learning_rate="invscaling",  max_iter=500,alpha=10)
mlp.fit(x_train, y_train)
pred_y_mlp = mlp.predict(x_test)
print('mlp 분류기 정확도', accuracy_score(y_test, pred_y_mlp))

mlp = MLPClassifier(hidden_layer_sizes=(512,128,32), activation='relu', batch_size=2500,
                    solver='adam', learning_rate="adaptive",  max_iter=500,alpha=1e-2,warm_start=True)
mlp.fit(x_train, y_train)
pred_y_mlp = mlp.predict(x_test)
print('mlp 분류기 정확도', accuracy_score(y_test, pred_y_mlp))'''

'''from sklearn.neural_network import MLPClassifier

score = np.zeros((3,))
for i in range(3):
  mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', batch_size=250,
                      solver='adam', learning_rate="adaptive",  max_iter=500,alpha=5e-1,random_state=i+1)
  mlp.fit(x_train, y_train)
  pred_y_mlp = mlp.predict(x_test)
  score[i]=accuracy_score(y_test, pred_y_mlp)
print(f'mlp 분류기 정확도:{np.average(score):.3f}')#0.791 5e-1'''



mlp 분류기 정확도:0.791


In [ ]:
from sklearn import svm

#svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1)

'''svm_clf = svm.SVC()

params = {
    'kernel': ['poly','rbf'],
    'gamma': ['auto',1e-2,1e-1,1e0,1e1,1e2],
    'C':[1e-2,1e-1,1e0,1e1,1e2],
    'tol':[1e-2,1e-1,1e0,1e1,1e-3],
    'probability':[True],
    'max_iter':[100,500,-1]
}

grid_tree = GridSearchCV(svm_clf, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, y_train)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
svm_clf = grid_tree.best_estimator_'''
'''svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1)#0.9878
svm_clf.fit(x_train, y_train)
pred_y_svm = svm_clf.predict(x_test)
print('svm_clf 분류기 정확도', accuracy_score(y_test, pred_y_svm))#Voting 분류기 정확도 0.99
#svm_clf.fit(x_train, corse_classify)
#pred_y_svm = svm_clf.predict(x_test)
svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=10,tol=1e-3)#0.9878
svm_clf.fit(x_train, y_train)
pred_y_svm = svm_clf.predict(x_test)
print('svm_clf 분류기 정확도', accuracy_score(y_test, pred_y_svm))#Voting 분류기 정확도 0.99'''
'''
best parameters :  {'C': 10.0, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'tol': 0.001}
best score :  0.7728098433618052
svm_clf 분류기 정확도 0.7948
'''
'''score = np.zeros((3,))
for i in range(3):
  svm_clf = svm.SVC(kernel='rbf',gamma='scale',probability=True,C=5e1,random_state=i+1)#0.9878
  svm_clf.fit(x_train, y_train)
  pred_y_mlp = svm_clf.predict(x_test)
  score[i]=accuracy_score(y_test, pred_y_mlp)
print(f'svm_clf 분류기 정확도:{np.average(score):.3f}')#0.799 c=1e2 or 5e1'''

'''score = np.zeros((3,))
for i in range(3):
  svm_clf = svm.SVC(kernel='poly',gamma='scale',probability=True,C=5e1,random_state=i+1)#0.9878
  svm_clf.fit(x_train, y_train)
  pred_y_mlp = svm_clf.predict(x_test)
  score[i]=accuracy_score(y_test, pred_y_mlp)
print(f'svm_clf 분류기 정확도:{np.average(score):.3f}')#0.788'''

svm_clf 분류기 정확도:0.787


In [ ]:
from sklearn.ensemble import VotingClassifier
print(x_test.shape)
for i in range(3):
  knn = KNeighborsClassifier(weights='distance',algorithm='auto',p=1,n_neighbors=5)
  mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', batch_size=250,
                      solver='adam', learning_rate="adaptive",  max_iter=500,alpha=5e-1,random_state=i+1)
  svm_clf1 = svm.SVC(kernel='rbf',gamma='scale',probability=True,C=5e1,random_state=i+1)
  svm_clf2 = svm.SVC(kernel='poly',gamma='auto',probability=True,C=1e1,tol=1e-1,random_state=i+1)

  vo_clf = VotingClassifier(estimators=[('knn',knn),('mlp', mlp),('svm1',svm_clf1),('svm2',svm_clf2)],  voting='hard')

  vo_clf.fit(x_train, y_train)
  pred_y = vo_clf.predict(x_test)
  print('corse 분류기 정확도', accuracy_score(y_test, pred_y))#Voting 분류기 정확도 0.7956

(2500, 324)
corse 분류기 정확도 0.8024
corse 분류기 정확도 0.8016
corse 분류기 정확도 0.8036
